##

### This is python notebook to explore the pages using opensource frame works

In [2]:
# Importing all the necessary libraries
# Import Fastai libraries
from fastai.vision.all import *

# Import PyTorch
import torch

# Import other necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
#import albumentations as A  # Add this import statement

In [3]:
#lets set the reproducibility of the code

def set_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Set seed for reproducibility
seed = 42
set_seed(seed)


In [4]:
# Step 1: Read and parse train.csv
train_df = pd.read_csv('../data/Train.csv')

In [5]:
## lets explore the train dataframe
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26156 entries, 0 to 26155
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   image_id     26156 non-null  object 
 1   bbox         23849 non-null  object 
 2   category_id  23849 non-null  float64
 3   id           23849 non-null  float64
dtypes: float64(2), object(2)
memory usage: 817.5+ KB


In [6]:
## lets drop all null rows particullary on category and bbox because its what we will use to train our model
# Remove rows with null values in 'bbox' or 'category_id'
train_df = train_df.dropna(subset=['bbox', 'category_id'])

In [7]:
## lets see the output of the data that we will be dealing with

train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23849 entries, 0 to 26155
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   image_id     23849 non-null  object 
 1   bbox         23849 non-null  object 
 2   category_id  23849 non-null  float64
 3   id           23849 non-null  float64
dtypes: float64(2), object(2)
memory usage: 931.6+ KB


In [8]:
# Step 2: Preprocess the data
def parse_bbox(bbox_str):
    # Assuming bbox_str is in the format '[x, y, width, height]'
    bbox_str = str(bbox_str)
    bbox = eval(bbox_str)
    return tuple(map(float, bbox))

train_df['bbox'] = train_df['bbox'].apply(parse_bbox)

In [9]:
## lets see after preprocessing
train_df.head()

,image_id,bbox,category_id,id
0,id_w55q2qr62fsk,"(122.0, 1.0, 42.0, 30.0)",2.0,6356.0
1,id_fvldv7o1kn9d,"(500.0, 141.0, 74.0, 70.0)",2.0,2305.0
2,id_5d1r9l1jp7b5,"(304.0, 525.0, 54.0, 58.0)",2.0,238.0
3,id_45qru79t6s4n,"(187.0, 298.0, 44.0, 56.0)",2.0,921.0
4,id_84cjf2pbqmtv,"(349.0, 721.0, 58.0, 59.0)",2.0,1362.0


In [10]:
class ObjectDetectionDataLoader:
    def __init__(self, df, path, image_col, bbox_col, label_col):
        self.df = df
        self.path = path
        self.image_col = image_col
        self.bbox_col = bbox_col
        self.label_col = label_col

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_id = self.df.iloc[idx]['image_id']
        image_path = self.path / f"{image_id}.tif"
        image = PILImage.create(image_path)
        bbox = eval(self.df.iloc[idx][self.bbox_col])  # Assuming bbox is a string representation of a list
        label = self.df.iloc[idx][self.label_col]
        return image, (bbox, label)

In [11]:
image_path = Path('../data/train/Images')

# Define column names for images, bounding boxes, and labels
image_col = 'image_id'
bbox_col = 'bbox'
label_col = 'category_id'

# Create dataset object
dataset = ObjectDetectionDataLoader(train_df, image_path, image_col, bbox_col, label_col)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [21]:
# model = vision_learner(dataloader, resnet50,n_out=None)
# Count the number of unique category IDs in the dataset
num_classes = len(train_df['category_id'].unique())

# Create a model
model =vision_learner(dataloader, resnet50, n_out=num_classes)


AttributeError: 'function' object has no attribute 'fs'